<a href="https://colab.research.google.com/github/jjsyeon/CodingTest/blob/main/Assignment/06_NLP_BasicProject/(%EB%AF%B8%EC%85%98_3)_K_fold.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# NLP 기초대회 미션 3 - K-fold 적용

**훈련 미션 : KLUE STS 데이터셋에 K-fold를 적용하여 모델의 성능 평가를 진행한다**

**미션 개요**
- K-fold를 통하여 모델의 성능 개선

**실습 배경 및 목적**
- STS 데이터셋에 pytorch lightning 코드 작성
- STS 데이터셋에 K-fold를 적용
- 일반 학습 방법과 K-fold 적용 방법의 성능 비교

**데이터셋(https://klue-benchmark.com/tasks/67/overview/description)**
- KLUE의 Semantic Textual Similarity(STS) 데이터셋
- 입력 : 두 문장
- 출력 : 두 문장의 유사도
- 학습 데이터 : 11,668개
- 검증 데이터 : 519개(평가 데이터가 비공개이므로 학습에서 평가데이터로 활용함)
- 평가 데이터 : 1,037개(비공개)

- License : <a rel="license" href="http://creativecommons.org/licenses/by-sa/4.0/">Creative Commons Attribution-ShareAlike 4.0 International License</a>.

<a rel="license" href="http://creativecommons.org/licenses/by-sa/4.0/"><img alt="Creative Commons License" style="border-width:0" src="https://i.creativecommons.org/l/by-sa/4.0/88x31.png" /></a><br />

**모델**
- [klue/roberta-small](https://huggingface.co/klue/roberta-small) 모델과 토크나이저 활용

# 환경 설정

In [ ]:
!pip install scikit-learn
!pip install transformers
!pip install pytorch_lightning

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 76.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 24.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 104.9 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 827.8/827.8 KB 33.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 518.6/518.6 KB 51.2 MB/s eta 0:00:00


In [ ]:
import os
import argparse
import requests

import pandas as pd
import json

from tqdm.auto import tqdm

import transformers
import torch
import torchmetrics
import pytorch_lightning as pl
from sklearn.model_selection import KFold

# 데이터셋

In [ ]:
class Dataset(torch.utils.data.Dataset):
    def __init__(self, inputs, targets=[]):
        self.inputs = inputs
        self.targets = targets

    # 학습 및 추론 과정에서 데이터를 1개씩 꺼내오는 곳
    def __getitem__(self, idx):
        # 정답이 있다면 if문을, 없다면 else문을 수행합니다
        if len(self.targets) == 0:
            return torch.tensor(self.inputs[idx])
        else:
            return torch.tensor(self.inputs[idx]), torch.tensor(self.targets[idx])

    # 입력하는 개수만큼 데이터를 사용합니다
    # 'return 100'이면 1에폭에 100개의 데이터만 사용합니다
    def __len__(self):
        return len(self.inputs)

In [ ]:
class Dataloader(pl.LightningDataModule):
    def __init__(self, model_name, batch_size, train_ratio, shuffle, bce):
        super().__init__()
        self.model_name = model_name
        self.batch_size = batch_size
        self.train_ratio = train_ratio
        self.shuffle = shuffle
        self.bce = bce

        self.train_dataset = None
        self.val_dataset = None
        self.test_dataset = None
        self.predict_dataset = None

        self.tokenizer = transformers.AutoTokenizer.from_pretrained(model_name, max_length=160)
        self.set_preprocessing()

    def read_json(self, data_type):
        # 파일이 없으면 다운로드 받아옵니다
        if not os.path.isfile(f"{data_type}.json"):
            data = requests.get(f"https://raw.githubusercontent.com/htw5295/klue_sts/main/klue-sts-v1.1_{data_type}.json")
            with open(f"{data_type}.json", 'w', encoding='utf-8') as f:
                json.dump(json.loads(data.text), f, ensure_ascii=False)

        # json 파일을 읽어 pandas 형태로 변환합니다
        with open(f'{data_type}.json', 'r', encoding='utf-8') as file:
            json_data = json.load(file)

        data = []
        for item in json_data:
            data.append([item['source'], item['sentence1'], item['sentence2'], item['labels']['label'], item['labels']['binary-label']])

        df = pd.DataFrame(data, columns=['source', 'sentence1', 'sentence2', 'label', 'binary_label'])

        return df

    def tokenizing(self, dataframe):
        data = []
        for idx, item in tqdm(dataframe.iterrows(), desc='tokenizing', total=len(dataframe)):
            # 두 입력 문장을 [SEP] 토큰으로 이어붙여서 전처리합니다.
            text = '[SEP]'.join([item[text_column] for text_column in self.text_columns])
            outputs = self.tokenizer(text, add_special_tokens=True, padding='max_length', truncation=True)
            data.append(outputs['input_ids'])

        return data

    def set_preprocessing(self):
        # 데이터를 읽어서 타겟 컬럼, 안쓰는 컬럼, 텍스트 전처리가 필요한 컬럼명을 기록합니다.
        data = self.read_json('train')
        columns = data.columns
        if self.bce:
            self.target_columns = [columns[4]]
        else:
            self.target_columns = [columns[3]]

        self.delete_columns = [columns[0]]
        self.text_columns = [columns[1], columns[2]]

    def preprocessing(self, data):
        # 안쓰는 컬럼을 삭제합니다.
        data = data.drop(columns=self.delete_columns)

        # 타겟 데이터가 없으면 빈 배열을 리턴합니다.
        try:
            targets = data[self.target_columns].values.tolist()
        except:
            targets = []
        # 텍스트 데이터를 전처리합니다.
        inputs = self.tokenizing(data)

        return inputs, targets

    def setup(self, stage='fit'):
        if stage == 'fit':
            total_data = self.read_json('train')

            # 학습 데이터와 검증 데이터셋을 비율에 맞춰 분리합니다
            train_data = total_data.sample(frac=self.train_ratio)
            val_data = total_data.drop(train_data.index)

            # 학습데이터 준비
            train_inputs, train_targets = self.preprocessing(train_data)

            # 검증데이터 준비
            val_inputs, val_targets = self.preprocessing(val_data)

            # train 데이터만 shuffle을 적용해줍니다, 필요하다면 val, test 데이터에도 shuffle을 적용할 수 있습니다
            self.train_dataset = Dataset(train_inputs, train_targets)
            self.val_dataset = Dataset(val_inputs, val_targets)
        else:
            # 평가데이터 준비
            test_data = self.read_json('dev')
            test_inputs, test_targets = self.preprocessing(test_data)
            self.test_dataset = Dataset(test_inputs, test_targets)
            self.predict_dataset = Dataset(test_inputs, [])

    def train_dataloader(self):
        return torch.utils.data.DataLoader(self.train_dataset, batch_size=self.batch_size, shuffle=args.shuffle)

    def val_dataloader(self):
        return torch.utils.data.DataLoader(self.val_dataset, batch_size=self.batch_size)

    def test_dataloader(self):
        return torch.utils.data.DataLoader(self.test_dataset, batch_size=self.batch_size)

    def predict_dataloader(self):
        return torch.utils.data.DataLoader(self.predict_dataset, batch_size=self.batch_size)

# K-fold가 되는 dataloader

In [ ]:
class KfoldDataloader(pl.LightningDataModule):
    def __init__(self, 
                 model_name, 
                 batch_size, 
                 shuffle, 
                 bce,
                 k: int = 1,  # fold number
                 split_seed: int = 12345,  # split needs to be always the same for correct cross validation
                 num_splits: int = 10):
        super().__init__()
        self.model_name = model_name
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.bce = bce
        self.k = k
        self.split_seed = split_seed
        self.num_splits = num_splits

        self.train_dataset = None
        self.val_dataset = None
        self.test_dataset = None
        self.predict_dataset = None

        self.tokenizer = transformers.AutoTokenizer.from_pretrained(model_name, max_length=160)
        self.set_preprocessing()

    def read_json(self, data_type):
        # 파일이 없으면 다운로드 받아옵니다
        if not os.path.isfile(f"{data_type}.json"):
            data = requests.get(f"https://raw.githubusercontent.com/htw5295/klue_sts/main/klue-sts-v1.1_{data_type}.json")
            with open(f"{data_type}.json", 'w', encoding='utf-8') as f:
                json.dump(json.loads(data.text), f, ensure_ascii=False)

        # json 파일을 읽어 pandas 형태로 변환합니다
        with open(f'{data_type}.json', 'r', encoding='utf-8') as file:
            json_data = json.load(file)

        data = []
        for item in json_data:
            data.append([item['source'], item['sentence1'], item['sentence2'], item['labels']['label'], item['labels']['binary-label']])

        df = pd.DataFrame(data, columns=['source', 'sentence1', 'sentence2', 'label', 'binary_label'])

        return df

    def tokenizing(self, dataframe):
        data = []
        for idx, item in tqdm(dataframe.iterrows(), desc='tokenizing', total=len(dataframe)):
            # 두 입력 문장을 [SEP] 토큰으로 이어붙여서 전처리합니다.
            text = '[SEP]'.join([item[text_column] for text_column in self.text_columns])
            outputs = self.tokenizer(text, add_special_tokens=True, padding='max_length', truncation=True)
            data.append(outputs['input_ids'])

        return data

    def set_preprocessing(self):
        # 데이터를 읽어서 타겟 컬럼, 안쓰는 컬럼, 텍스트 전처리가 필요한 컬럼명을 기록합니다.
        data = self.read_json('train')
        columns = data.columns
        if self.bce:
            self.target_columns = [columns[4]]
        else:
            self.target_columns = [columns[3]]

        self.delete_columns = [columns[0]]
        self.text_columns = [columns[1], columns[2]]

    def preprocessing(self, data):
        # 안쓰는 컬럼을 삭제합니다.
        data = data.drop(columns=self.delete_columns)

        # 타겟 데이터가 없으면 빈 배열을 리턴합니다.
        try:
            targets = data[self.target_columns].values.tolist()
        except:
            targets = []
        # 텍스트 데이터를 전처리합니다.
        inputs = self.tokenizing(data)

        return inputs, targets

    def setup(self, stage='fit'):
        if stage == 'fit':
            # 데이터 준비
            total_data = self.read_json('train')
            total_input, total_targets = self.preprocessing(total_data)
            total_dataset = Dataset(total_input, total_targets)

            # 데이터셋 num_splits 번 fold
            kf = KFold(n_splits=self.num_splits, shuffle=self.shuffle, random_state=self.split_seed)
            all_splits = [k for k in kf.split(total_dataset)]
            # k번째 fold 된 데이터셋의 index 선택
            train_indexes, val_indexes = all_splits[self.k]
            train_indexes, val_indexes = train_indexes.tolist(), val_indexes.tolist()

            # fold한 index에 따라 데이터셋 분할
            self.train_dataset = [total_dataset[x] for x in train_indexes] 
            self.val_dataset = [total_dataset[x] for x in val_indexes]

        else:
            # 평가데이터 준비
            test_data = self.read_json('dev')
            test_inputs, test_targets = self.preprocessing(test_data)
            self.test_dataset = Dataset(test_inputs, test_targets)
            self.predict_dataset = Dataset(test_inputs, [])

    def train_dataloader(self):
        return torch.utils.data.DataLoader(self.train_dataset, batch_size=self.batch_size, shuffle=self.shuffle)

    def val_dataloader(self):
        return torch.utils.data.DataLoader(self.val_dataset, batch_size=self.batch_size)

    def test_dataloader(self):
        return torch.utils.data.DataLoader(self.test_dataset, batch_size=self.batch_size)

    def predict_dataloader(self):
        return torch.utils.data.DataLoader(self.predict_dataset, batch_size=self.batch_size)

# test 성능 결과가 출력되는 모델

In [ ]:
class Model(pl.LightningModule):
    def __init__(self, model_name, lr, bce):
        super().__init__()
        self.save_hyperparameters()

        self.model_name = model_name
        self.lr = lr
        self.bce = bce

        self.plm = transformers.AutoModelForSequenceClassification.from_pretrained(pretrained_model_name_or_path=model_name, num_labels=1)
        if self.bce:
            self.loss_func = torch.nn.BCEWithLogitsLoss()
            self.evaluation = torchmetrics.F1Score(task='binary')
        else:
            self.loss_func = torch.nn.L1Loss()
            self.evaluation = torchmetrics.PearsonCorrCoef()

    def forward(self, x):
        x = self.plm(x)['logits']

        return x

    def training_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        loss = self.loss_func(logits, y.float())
        self.log("train_loss", loss)

        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        loss = self.loss_func(logits, y.float())
        self.log("val_loss", loss)
        if self.bce:
            self.log("val_f1", self.evaluation(logits, y))
        else:
            self.log("val_pearson", self.evaluation(logits, y))

        return loss

    def test_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        if self.bce:
            self.log("test_f1", self.evaluation(logits, y))
        else:
            self.log("test_pearson", self.evaluation(logits, y))
        return self.evaluation(logits, y)

    def predict_step(self, batch, batch_idx):
        x = batch
        logits = self(x)

        return logits

    def configure_optimizers(self):
        # 이곳에서 lr 값을 변경할 수 있습니다
        optimizer = torch.optim.AdamW(self.parameters(), lr=self.lr)
        return optimizer

# 일반 학습 평가

In [ ]:
# 하이퍼 파라미터 등 각종 설정값을 입력받습니다
# 터미널 실행 예시 : python3 run.py --batch_size=64 ...
# 실행 시 '--batch_size=64' 같은 인자를 입력하지 않으면 default 값이 기본으로 실행됩니다
parser = argparse.ArgumentParser()
parser.add_argument('--model_name', default='klue/roberta-small', type=str)
parser.add_argument('--batch_size', default=16, type=int)
parser.add_argument('--max_epoch', default=30, type=int)
parser.add_argument('--shuffle', default=True)
parser.add_argument('--bce', default=True)
parser.add_argument('--train_ratio', default=0.8)
parser.add_argument('--learning_rate', default=1e-5, type=float)
args = parser.parse_args(args=[])

# 기본 Dataloader
dataloader = Dataloader(args.model_name, args.batch_size, args.train_ratio, args.shuffle, args.bce)
# 성능 비교를 위한 모델 생성
model= Model(args.model_name, args.learning_rate, args.bce)

# K fold의 동일한 횟수의 epoch 학습 (3)
trainer = pl.Trainer(max_epochs=3)
trainer.fit(model=model, datamodule=dataloader)
trainer.test(model=model, datamodule=dataloader)

Some weights of the model checkpoint at klue/roberta-small were not used when initializing RobertaForSequenceClassification: ['lm_head.layer_norm.weight', 'lm_head.decoder.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.weight', 'lm_head.bias', 'lm_head.decoder.bias', 'lm_head.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at klue/roberta-small and are newly initialized: ['classifier.out_proj.bias', 'classifier.dense.bias', 'classif

tokenizing:   0%|          | 0/9334 [00:00<?, ?it/s]

tokenizing:   0%|          | 0/2334 [00:00<?, ?it/s]

INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name       | Type                             | Params
----------------------------------------------------------------
0 | plm        | RobertaForSequenceClassification | 68.1 M
1 | loss_func  | BCEWithLogitsLoss                | 0     
2 | evaluation | BinaryF1Score                    | 0     
----------------------------------------------------------------
68.1 M    Trainable params
0         Non-trainable params
68.1 M    Total params
272.367   Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

...

tokenizing:   0%|          | 0/519 [00:00<?, ?it/s]

INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: 0it [00:00, ?it/s]

────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
         test_f1            0.7851788997650146
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


[{'test_f1': 0.7851788997650146}]

# K-fold 학습 평가

In [ ]:
# 새 모델 생성
Kmodel = Model(args.model_name, args.learning_rate, args.bce)

results = []
# K fold 횟수 3
nums_folds = 3
split_seed = 12345

# nums_folds는 fold의 개수, k는 k번째 fold datamodule
for k in range(nums_folds):
    datamodule = KfoldDataloader(args.model_name, args.batch_size, args.shuffle, args.bce, 
                                k=k, split_seed=split_seed, num_splits=nums_folds)
    datamodule.prepare_data()
    datamodule.setup()

    trainer = pl.Trainer(max_epochs=1)
    trainer.fit(model=model, datamodule=dataloader)
    score = trainer.test(model=model, datamodule=dataloader)

    results.extend(score)

Some weights of the model checkpoint at klue/roberta-small were not used when initializing RobertaForSequenceClassification: ['lm_head.layer_norm.weight', 'lm_head.decoder.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.weight', 'lm_head.bias', 'lm_head.decoder.bias', 'lm_head.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at klue/roberta-small and are newly initialized: ['classifier.out_proj.bias', 'classifier.dense.bias', 'classif

tokenizing:   0%|          | 0/11668 [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


tokenizing:   0%|          | 0/9334 [00:00<?, ?it/s]

tokenizing:   0%|          | 0/2334 [00:00<?, ?it/s]

...

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=1` reached.


tokenizing:   0%|          | 0/519 [00:00<?, ?it/s]

INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: 0it [00:00, ?it/s]

────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
         test_f1            0.7985547184944153
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


In [ ]:
# 모델의 평균 성능

if args.bce:
    result = [x['test_f1'] for x in results]
    score = sum(result) / nums_folds
    print("K fold Test f1 score: ", score)
else:
    result = [x['test_pearson'] for x in results]
    score = sum(result) / nums_folds
    print("K fold Test pearson: ", score)

K fold Test f1 score:  0.795535147190094


###**콘텐츠 라이선스**

<font color='red'><b>**WARNING**</b></font> : **본 교육 콘텐츠의 지식재산권은 재단법인 네이버커넥트에 귀속됩니다. 본 콘텐츠를 어떠한 경로로든 외부로 유출 및 수정하는 행위를 엄격히 금합니다.** 다만, 비영리적 교육 및 연구활동에 한정되어 사용할 수 있으나 재단의 허락을 받아야 합니다. 이를 위반하는 경우, 관련 법률에 따라 책임을 질 수 있습니다.